# Etiquetamento Autônomo de Perguntas do StackOverflow com Deep Learning

- Bruno Pilão
- Maria Mello
- Larissa Nobrega
- Fernanda Moyses 


O objetivo do trabalho é desenvolver um modelo Deep Learning para o etiquetamento autônomo (*multi-label classification*) de perguntas do StackOverflow.

CONJUNTO DE DADOS

A base de dados a ser utilizada é a StackLite, disponível em https://github.com/dgrtwo/StackLite.

A StackLite é uma versão simplificada e pré-processada de uma parte dos dados do Stack Overflow, contendo perguntas e suas tags associadas.

Embora o repositório possa conter outros arquivos, o foco será nos dados que permitem mapear perguntas (corpo e/ou título) para suas tags.

Obs.: Uma única pergunta pode ter múltiplas tags, o que a torna um problema de classificação multilabel. Pense em uma forma de priorizar uma das tags para simplificar o problema.



In [2]:
# Importando Bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import MultiLabelBinarizer,StandardScaler,LabelEncoder
from sklearn.metrics import classification_report,hamming_loss
import ast
import seaborn as sns
import matplotlib.pyplot as plt


## Análise breve dataset 


### Dataset Auxiliar 

Este dataset constitui um dicionário que mapeia cada questão gerada no Stack Overflow às suas respectivas categorias temáticas. O conjunto de dados auxiliar será utilizado para realizar uma análise prévia com o objetivo de avaliar e classificar os tipos de respostas associadas a cada pergunta.

In [3]:
#Importação dataset tags
df_tags = pd.read_csv('data\question_tags.csv.gz',compression='gzip')
df_tags.head()

,Id,Tag
0,1,data
1,4,c#
2,4,winforms
3,4,type-conversion
4,4,decimal


In [4]:
#Validandop quantidade de itens na datable
len(df_tags)

52224835

In [5]:
#Validando tipos de tag
print("Quantidade de itens por ID")
for i in range(10):
    print(i+1,df_tags[df_tags['Id'] == i+1]['Tag'].values)

Quantidade de itens por ID
1 ['data']
2 []
3 []
4 ['c#' 'winforms' 'type-conversion' 'decimal' 'opacity']
5 []
6 ['html' 'css' 'css3' 'internet-explorer-7']
7 []
8 ['c#' 'code-generation' 'j#' 'visualj#']
9 ['c#' '.net' 'datetime']
10 []


Observamos que as tags não possuem um ordem sequencial e podem conter um ou multiplos valores dentro de um ID.

In [6]:
# GroupBy simples: ID → lista de tags
grouped_mini = df_tags.groupby('Id')['Tag'].apply(list).reset_index()

print(f"✅ Grouped criado: {len(grouped_mini):,} IDs únicos")

✅ Grouped criado: 17,763,486 IDs únicos


In [7]:

# Salvar em CSV ()
grouped_mini.to_csv('stackoverflow_grouped_tags.csv', index=False)
print("✅ Backup CSV: stackoverflow_grouped_tags.csv")

✅ Backup CSV: stackoverflow_grouped_tags.csv


In [8]:
# Comprimir CSV (reduz ~80%)
df = pd.read_csv('stackoverflow_grouped_tags.csv')
df.to_csv('stackoverflow_grouped_tags.csv.gz', compression='gzip', index=False)

Com isso avaliamos que este dataset serve como um "Dicionário" para identificar o tipo de pergunta que o usuario realizou no stackoverflow, sendo que uma tg pode contem um ou múltiplos assuntos envolvidos.


### Dataset Principal

O dataset principal consiste em perguntas técnicas geradas por usuários da plataforma Stack Overflow. Este conjunto de dados contém questões formuladas por desenvolvedores e profissionais de tecnologia que buscam soluções para problemas específicos em suas áreas de atuação.

In [9]:
#Importação dataset questions
df_questions = pd.read_csv('data\questions.csv.gz',compression='gzip')
df_questions.head()

,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount
0,1,2008-07-31T21:26:37Z,NaN,2011-03-28T00:53:47Z,1,NaN,0.0
1,4,2008-07-31T21:42:52Z,NaN,NaN,472,8.0,13.0
2,6,2008-07-31T22:08:08Z,NaN,NaN,210,9.0,5.0
3,8,2008-07-31T23:33:19Z,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0
4,9,2008-07-31T23:40:59Z,NaN,NaN,1452,1.0,58.0


In [10]:
# Quantiade de itens df_qustions

len(df_questions)

17763486

In [11]:
# Validar tipos de variáveis ao nosso dataset
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17763486 entries, 0 to 17763485
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Id            int64  
 1   CreationDate  object 
 2   ClosedDate    object 
 3   DeletionDate  object 
 4   Score         int64  
 5   OwnerUserId   float64
 6   AnswerCount   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 948.7+ MB


## Merge Datasets

Vamos agrupar nosso dataset para melhor perfomace no treinamento do nosso modelo para predição de tag do stack overflow

In [12]:
# Vamos realizar um merge de nosso dataset 
df = df_questions.merge(grouped_mini,on='Id',how='inner')

# Verificar resultado
print(f"Dataset final: {df.shape}")
df.head()

Dataset final: (17763486, 8)


,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount,Tag
0,1,2008-07-31T21:26:37Z,NaN,2011-03-28T00:53:47Z,1,NaN,0.0,[data]
1,4,2008-07-31T21:42:52Z,NaN,NaN,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]"
2,6,2008-07-31T22:08:08Z,NaN,NaN,210,9.0,5.0,"[html, css, css3, internet-explorer-7]"
3,8,2008-07-31T23:33:19Z,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0,"[c#, code-generation, j#, visualj#]"
4,9,2008-07-31T23:40:59Z,NaN,NaN,1452,1.0,58.0,"[c#, .net, datetime]"


In [13]:
# Converter para datetime
df['CreationDate'] = pd.to_datetime(df['CreationDate'])
df['Year'] = df['CreationDate'].dt.year
df['Month'] = df['CreationDate'].dt.month  
df['Day'] = df['CreationDate'].dt.day

In [14]:
df.head()

,Id,CreationDate,ClosedDate,DeletionDate,Score,OwnerUserId,AnswerCount,Tag,Year,Month,Day
0,1,2008-07-31 21:26:37+00:00,NaN,2011-03-28T00:53:47Z,1,NaN,0.0,[data],2008,7,31
1,4,2008-07-31 21:42:52+00:00,NaN,NaN,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]",2008,7,31
2,6,2008-07-31 22:08:08+00:00,NaN,NaN,210,9.0,5.0,"[html, css, css3, internet-explorer-7]",2008,7,31
3,8,2008-07-31 23:33:19+00:00,2013-06-03T04:00:25Z,2015-02-11T08:26:40Z,42,NaN,8.0,"[c#, code-generation, j#, visualj#]",2008,7,31
4,9,2008-07-31 23:40:59+00:00,NaN,NaN,1452,1.0,58.0,"[c#, .net, datetime]",2008,7,31


In [15]:
# Removendo colunas que não são necessarios para nosso treinamento
df = df.drop(columns=["ClosedDate","DeletionDate","CreationDate","Id"],axis=1)

# Validando DataFrame
df.head()

,Score,OwnerUserId,AnswerCount,Tag,Year,Month,Day
0,1,NaN,0.0,[data],2008,7,31
1,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]",2008,7,31
2,210,9.0,5.0,"[html, css, css3, internet-explorer-7]",2008,7,31
3,42,NaN,8.0,"[c#, code-generation, j#, visualj#]",2008,7,31
4,1452,1.0,58.0,"[c#, .net, datetime]",2008,7,31


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17763486 entries, 0 to 17763485
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Score        int64  
 1   OwnerUserId  float64
 2   AnswerCount  float64
 3   Tag          object 
 4   Year         int32  
 5   Month        int32  
 6   Day          int32  
dtypes: float64(2), int32(3), int64(1), object(1)
memory usage: 745.4+ MB


Com isso vamos utilizar este dataframe para realizar uma predição com uma frase de texto bruta e acimiliar o tipo de conteudo que estou relatando

## Treinamento de Modelo

Vamos inicialmente ajustar o nosso treinamento do modelo para que com uma frase de entrada o nosso modelo consiga captar 

In [17]:
df.head()

,Score,OwnerUserId,AnswerCount,Tag,Year,Month,Day
0,1,NaN,0.0,[data],2008,7,31
1,472,8.0,13.0,"[c#, winforms, type-conversion, decimal, opacity]",2008,7,31
2,210,9.0,5.0,"[html, css, css3, internet-explorer-7]",2008,7,31
3,42,NaN,8.0,"[c#, code-generation, j#, visualj#]",2008,7,31
4,1452,1.0,58.0,"[c#, .net, datetime]",2008,7,31


In [18]:
# Reduz para uma amostra da base mantendo a distribuição das tags
df_sampled = df.sample(n=50_000, random_state=42).copy()

In [19]:
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from collections import Counter

# Corrigir a coluna de tags
def safe_literal_eval(tag_value):
    try:
        return ast.literal_eval(tag_value)
    except:
        try:
            tag_str = str(tag_value).strip()
            if tag_str in ['nan', 'None', '', '[]']:
                return []
            tag_str = tag_str.strip('[]')
            tags = [tag.strip().strip("'").strip('"') for tag in tag_str.split(',')]
            return [tag for tag in tags if tag]
        except:
            return []


In [20]:
# Avaliar e limpar os dados
df['Tag'] = df['Tag'].apply(safe_literal_eval)
df = df[df['Tag'].map(len) > 0]

# Contar frequência de todas as tags
all_tags = [tag for tags in df['Tag'] for tag in tags]
tag_counts = Counter(all_tags)

# Definir o número de tags mais comuns que você quer manter 
top_tags = set([tag for tag, count in tag_counts.most_common(50)])

# Filtrar somente linhas com pelo menos uma tag entre as mais comuns
df['Tag'] = df['Tag'].apply(lambda tags: [tag for tag in tags if tag in top_tags])
df = df[df['Tag'].map(len) > 0]  # Remover quem ficou com lista vazia

In [21]:
features = ["Year", "Month", "Day", "Score", "OwnerUserId", "AnswerCount"]
X = df[features]

mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['Tag'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f'Treinamento: {X_train.shape}')
print(f'Teste: {X_test.shape}')
print(f'Número de classes: {len(mlb.classes_)}')


Treinamento: (10814617, 6)
Teste: (2703655, 6)
Número de classes: 50


In [25]:
# ===================================================================
# DIAGNÓSTICO + PREPARAÇÃO DOS DADOS ADAPTADO AO SEU SETUP
# ===================================================================

import tensorflow as tf
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score, hamming_loss, accuracy_score

# VERIFICAR PROBLEMA DAS CLASSES
print("=== DIAGNÓSTICO DO PROBLEMA ===")
print(f'Treinamento: {X_train.shape}')
print(f'Teste: {X_test.shape}')
print(f'Número de classes: {len(mlb.classes_)}')

# Se muitas classes, vamos reduzir
if len(mlb.classes_) > 100:
    print(f"⚠️  MUITAS CLASSES ({len(mlb.classes_)})! Reduzindo para top 50...")
    
    # Contar frequência das labels
    label_counts = y_train.sum(axis=0)
    top_50_indices = np.argsort(label_counts)[-50:][::-1]
    
    # Filtrar apenas top 50
    y_train = y_train[:, top_50_indices]
    y_test = y_test[:, top_50_indices]
    
    # Atualizar classes
    top_classes = [mlb.classes_[i] for i in top_50_indices]
    print(f"Reduzido para {y_train.shape[1]} classes")
    print(f"Top 10 classes: {top_classes[:10]}")
else:
    print(f"✅ Número de classes OK: {len(mlb.classes_)}")

# Normalizar dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.fillna(0))
X_test_scaled = scaler.transform(X_test.fillna(0))

print(f"Dataset preparado: {X_train_scaled.shape}")
print(f"Labels shape: {y_train.shape}")

# ===================================================================
# ARQUITETURA 1: CNN SIMPLES
# ===================================================================

def create_simple_cnn():
    """CNN Simples para baseline"""
    
    # Reshape para CNN: (samples, features, channels)
    X_train_cnn = X_train_scaled.reshape(-1, 6, 1)
    X_test_cnn = X_test_scaled.reshape(-1, 6, 1)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(6, 1)),
        
        # CNN Simples
        tf.keras.layers.Conv1D(32, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.2),
        
        tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.3),
        
        # Global pooling
        tf.keras.layers.GlobalAveragePooling1D(),
        
        # Dense layers
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        
        # Output multilabel - USAR TAMANHO CORRETO
        tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')
    ], name='CNN_Simples')
    
    return model, X_train_cnn, X_test_cnn

# ===================================================================
# ARQUITETURA 2: CNN COMPLEXA (INCEPTION-LIKE)
# ===================================================================

def inception_block_1d(x, f1, f3_red, f3, f5_red, f5, pool_proj):
    """Bloco Inception simplificado"""
    # Branch 1: 1x1
    branch1 = tf.keras.layers.Conv1D(f1, 1, activation='relu', padding='same')(x)
    
    # Branch 2: 1x1 -> 3x3
    branch2 = tf.keras.layers.Conv1D(f3_red, 1, activation='relu', padding='same')(x)
    branch2 = tf.keras.layers.Conv1D(f3, 3, activation='relu', padding='same')(branch2)
    
    # Branch 3: 1x1 -> 5x5 (substituído por duas 3x3)
    branch3 = tf.keras.layers.Conv1D(f5_red, 1, activation='relu', padding='same')(x)
    branch3 = tf.keras.layers.Conv1D(f5, 3, activation='relu', padding='same')(branch3)
    
    # Branch 4: pool -> 1x1
    branch4 = tf.keras.layers.MaxPooling1D(3, strides=1, padding='same')(x)
    branch4 = tf.keras.layers.Conv1D(pool_proj, 1, activation='relu', padding='same')(branch4)
    
    return tf.keras.layers.Concatenate()([branch1, branch2, branch3, branch4])

def create_complex_cnn():
    """CNN Complexa com Inception Blocks"""
    
    inputs = tf.keras.layers.Input(shape=(6, 1))
    
    # Entrada
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # Inception blocks
    x = inception_block_1d(x, 16, 24, 32, 8, 16, 8)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    
    x = inception_block_1d(x, 32, 48, 64, 16, 32, 16)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    x = inception_block_1d(x, 64, 96, 128, 32, 64, 32)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    # Global pooling
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    
    # Dense layers mais robustas
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    
    # Output - USAR TAMANHO CORRETO
    outputs = tf.keras.layers.Dense(y_train.shape[1], activation='sigmoid')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='CNN_Complexa_Inception')
    return model

# ===================================================================
# TREINAMENTO E COMPARAÇÃO
# ===================================================================

def train_and_compare():
    """Treina ambos modelos e compara performance"""
    
    # Criar modelos
    print("=== CRIANDO MODELOS ===")
    model_simple, X_train_cnn, X_test_cnn = create_simple_cnn()
    model_complex = create_complex_cnn()
    
    # Compilar ambos
    compile_config = {
        'optimizer': 'adam',
        'loss': 'binary_crossentropy',
        'metrics': ['accuracy', 'precision', 'recall']
    }
    
    model_simple.compile(**compile_config)
    model_complex.compile(**compile_config)
    
    print(f"Modelo Simples: {model_simple.count_params():,} parâmetros")
    print(f"Modelo Complexo: {model_complex.count_params():,} parâmetros")
    
    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)
    ]
    
    # Treinar modelo simples
    print("\n=== TREINANDO CNN SIMPLES ===")
    history_simple = model_simple.fit(
        X_train_cnn, y_train,
        validation_data=(X_test_cnn, y_test),
        epochs=20,  # Reduzido para testar
        batch_size=256,
        callbacks=callbacks,
        verbose=1
    )
    
    # Treinar modelo complexo
    print("\n=== TREINANDO CNN COMPLEXA ===")
    history_complex = model_complex.fit(
        X_train_cnn, y_train,
        validation_data=(X_test_cnn, y_test),
        epochs=20,  # Reduzido para testar
        batch_size=256,
        callbacks=callbacks,
        verbose=1
    )
    
    return model_simple, model_complex, history_simple, history_complex, X_test_cnn

# ===================================================================
# AVALIAÇÃO RÁPIDA
# ===================================================================

def evaluate_models_quick(model_simple, model_complex, X_test_cnn, y_test):
    """Avaliação rápida dos modelos"""
    
    print("=== AVALIAÇÃO FINAL DOS MODELOS ===")
    
    # Predições
    y_pred_simple = model_simple.predict(X_test_cnn)
    y_pred_complex = model_complex.predict(X_test_cnn)
    
    # Converter para binário (threshold=0.5)
    y_pred_simple_bin = (y_pred_simple > 0.5).astype(int)
    y_pred_complex_bin = (y_pred_complex > 0.5).astype(int)
    
    # Métricas
    metrics = {
        'CNN Simples': {
            'Hamming Loss': hamming_loss(y_test, y_pred_simple_bin),
            'F1-Score (micro)': f1_score(y_test, y_pred_simple_bin, average='micro'),
            'F1-Score (macro)': f1_score(y_test, y_pred_simple_bin, average='macro'),
            'Accuracy': accuracy_score(y_test, y_pred_simple_bin)
        },
        'CNN Complexa': {
            'Hamming Loss': hamming_loss(y_test, y_pred_complex_bin),
            'F1-Score (micro)': f1_score(y_test, y_pred_complex_bin, average='micro'),
            'F1-Score (macro)': f1_score(y_test, y_pred_complex_bin, average='macro'),
            'Accuracy': accuracy_score(y_test, y_pred_complex_bin)
        }
    }
    
    # Tabela comparativa
    import pandas as pd
    df_metrics = pd.DataFrame(metrics).T
    print("\n📊 Comparação de Métricas:")
    print(df_metrics.round(4))
    
    return df_metrics

# ===================================================================
# EXECUTAR TUDO
# ===================================================================

print("🚀 Iniciando comparação de arquiteturas...")

# Treinar modelos
model_simple, model_complex, history_simple, history_complex, X_test_cnn = train_and_compare()

# Avaliação final
metrics_df = evaluate_models_quick(model_simple, model_complex, X_test_cnn, y_test)

print("✅ Análise completa!")

=== DIAGNÓSTICO DO PROBLEMA ===
Treinamento: (10814617, 6)
Teste: (2703655, 6)
Número de classes: 50
✅ Número de classes OK: 50
Dataset preparado: (10814617, 6)
Labels shape: (10814617, 50)
🚀 Iniciando comparação de arquiteturas...
=== CRIANDO MODELOS ===
Modelo Simples: 26,546 parâmetros
Modelo Complexo: 223,018 parâmetros

=== TREINANDO CNN SIMPLES ===
Epoch 1/20
42245/42245 ━━━━━━━━━━━━━━━━━━━━ 221s 5ms/step - accuracy: 0.0822 - loss: 0.1265 - precision: 0.0338 - recall: 0.0019 - val_accuracy: 0.0844 - val_loss: 0.1202 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/20
17110/42245 ━━━━━━━━━━━━━━━━━━━━ 1:56 5ms/step - accuracy: 0.0892 - loss: 0.1209 - precision: 0.0000e+00 - recall: 0.0000e+00

KeyboardInterrupt: 